<h1 style="color:#27469c"> Preprocessing of ESB Cooling Towers 1 and 2 data </h1>


<h1 style="color:#27469c">ESB Cooling Tower 1</h1>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import neighbors
from mlxtend import plotting
import preprocessor

import warnings
warnings.filterwarnings('ignore')

datapath = "../data"

In [2]:
# load cooling tower data
df = pd.read_csv(f'{datapath}/esb/esb_cooling_tower_1.csv', index_col='time')
df.index = pd.to_datetime(df.index, utc='True')

initial_data_size = df.shape[0]

<h3 style="color:#27469c">Missing data removal</h3>

In [3]:
df = preprocessor.remove_missing_data(datadf=df, initial_data_size=initial_data_size, threshold=0.1)

After missing data removal, we are left with us with 104028 rows out of 104976.


In [4]:
df.columns

Index(['Cell_1bFan fanOnOff', 'Cell_1bFan fanStatus', 'Cell_1bFan vfdPercent',
       'Cell_1bFan vfdPower', 'Cell_1aFan fanOnOff', 'Cell_1aFan fanStatus',
       'Cell_1aFan vfdPercent', 'Cell_1aFan vfdPower',
       'ESB_Tower_1 bypassValveOpenClose', 'ESB_Tower_1 coolingTowerEnable',
       'ESB_Tower_1 enteringWaterTemp', 'ESB_Tower_1 leavingWaterTemp',
       'ESB_Tower_1 outdoorAirDryBulb', 'ESB_Tower_1 outdoorAirHumidity',
       'ESB_Tower_1 outdoorAirWetBulb'],
      dtype='object')

<h3 style="color:#27469c">Convert to SI Units</h3>
This may not be required for the transfer learning problem unless I'm doing data imputation (which I haven't needed to do yet.)

If needed, follow this code:

```
from bdx import apply_heuristics

df, left = apply_heuristics(df)
print('The following fields did not match and were not converted to SI units:', left)
```


<h3 style="color:#27469c">Outliers</h3>

In [5]:
df = preprocessor.remove_outliers_std(df, threshold=5, has_off_data=True, on_condition=df['Cell_1aFan fanOnOff'] | df['Cell_1bFan fanOnOff'])

Cell_1bFan vfdPercent has 0 outliers
Cell_1bFan vfdPower has 0 outliers
Cell_1aFan vfdPercent has 0 outliers
Cell_1aFan vfdPower has 0 outliers
ESB_Tower_1 enteringWaterTemp has 0 outliers
ESB_Tower_1 leavingWaterTemp has 0 outliers
ESB_Tower_1 outdoorAirDryBulb has 10 outliers
ESB_Tower_1 outdoorAirHumidity has 0 outliers
ESB_Tower_1 outdoorAirWetBulb has 36 outliers
Outier removal removed 46 rows (0.04421886415195909% of data) with outliers. Now left with 103982 rows.


<h3 style="color:#27469c">Add efficiency and season columns</h3>

In [6]:
preprocessor.create_efficiency_col(datadf=df, enteringWaterTemp="ESB_Tower_1 enteringWaterTemp", leavingWaterTemp='ESB_Tower_1 leavingWaterTemp', outdoorAirWetBulb='ESB_Tower_1 outdoorAirWetBulb', efficiency_col_name='ESB_Tower_1 efficiency')

preprocessor.create_season_col(datadf=df, season_col_name="ESB_Tower_1 season")

In [7]:
# save preprocessed tower 1 data
df.to_csv(f'{datapath}/esb/esb_tower_1_preprocessed.csv')

<h1 style="color:#27469c">ESB Cooling Tower 2</h1>

Replicate the same steps as for cooling tower 1

In [8]:
# load cooling tower 2 data
df = pd.read_csv(f'{datapath}/esb/esb_cooling_tower_2.csv', index_col='time')
df.index = pd.to_datetime(df.index, utc='True')

initial_data_size = df.shape[0]

# missing data removal
df = preprocessor.remove_missing_data(datadf=df, initial_data_size=initial_data_size, threshold=0.1)

# outlier removal
df = preprocessor.remove_outliers_std(df, threshold=5, has_off_data=True, on_condition=df['Cell_2aFan fanOnOff'] | df['Cell_2bFan fanOnOff'])

# add efficiency and season column
preprocessor.create_efficiency_col(datadf=df, enteringWaterTemp="ESB_Tower_2 enteringWaterTemp", leavingWaterTemp='ESB_Tower_2 leavingWaterTemp', outdoorAirWetBulb='ESB_Tower_2 outdoorAirWetBulb', efficiency_col_name='ESB_Tower_2 efficiency')
preprocessor.create_season_col(datadf=df, season_col_name="ESB_Tower_2 season")

# save preprocessed tower 2 data
df.to_csv(f'{datapath}/esb/esb_tower_2_preprocessed.csv')

After missing data removal, we are left with us with 104028 rows out of 104976.
Cell_2bFan vfdPercent has 0 outliers
Cell_2bFan vfdPower has 0 outliers
Cell_2aFan vfdPercent has 0 outliers
Cell_2aFan vfdPower has 0 outliers
ESB_Tower_2 enteringWaterTemp has 0 outliers
ESB_Tower_2 leavingWaterTemp has 0 outliers
ESB_Tower_2 outdoorAirDryBulb has 10 outliers
ESB_Tower_2 outdoorAirHumidity has 0 outliers
ESB_Tower_2 outdoorAirWetBulb has 36 outliers
Outier removal removed 46 rows (0.04421886415195909% of data) with outliers. Now left with 103982 rows.


<h2 style="color:#27469c">Merging data from cooling tower 2 into cooling tower 1 for cases where 2 was on and 1 was off - DON'T DO THIS</h2>

<h3 style="color:#27469c">Context</h3>

For the Engineering Science Building, typically either cooling tower is turned on at a time and not both. So in case denser data is needed, we could potentially consolidate data from both cooling towers into a singular dataset, such that data for times when cooling tower 1 was off can be filled in by data in cooling tower 2. The analysis done in the KNN plots below was done to visualize if the two cooling towers operate similarly.<br/>

Previously, this approach of combining the datasets was followed after reaching the conclusion that the behavior of the cooling towers is mostly similar.<br/>
However, based on the plots it seems that it would be better to treat the two cooling towers separately for model development purposes. The combined data will still be used to identify common variables across multiple buildings.

```
df = pd.read_csv(f'{datapath}/esb/esb_tower_1_preprocessed.csv', index_col="time").merge(pd.read_csv(f'{datapath}/esb/esb_tower_2_preprocessed.csv', index_col="time"), on="time")
df.index = pd.to_datetime(df.index, utc='True')
```

<h3 style="color:#27469c">Remove rows of data in which both cooling towers were off</h3>

```
# Number of rows in which neither cooling towers was on - 2078
# Remove these rows
df = df[df['Cell_1aFan fanOnOff'] & df['Cell_1bFan fanOnOff'] | df['Cell_2aFan fanOnOff'] & df['Cell_2bFan fanOnOff']]

# Number of rows in which both cooling towers were on - 6
# Since there are only 6 rows, we can remove these rows
df = df[(df['Cell_1aFan fanOnOff'] & df['Cell_1bFan fanOnOff']) ^ (df['Cell_2aFan fanOnOff'] & df['Cell_2bFan fanOnOff'])]
```

<h3 style="color:#27469c">Merge cooling tower 1 with cooling tower 2 data</h3>

```
# Assign values from Cell_2a columns to Cell_1a columns wherever cooling tower 1 was off
# Do not change Cell_1aFan fanOnOff and Cell_2aFan fanOnOff data

condition = df['Cell_1aFan fanOnOff'] == False

df.loc[df['Cell_1aFan fanOnOff'] == True, 'whichCoolingTower'] = 1
df.loc[condition, 'whichCoolingTower'] = 2

for column_name in df.columns:
    if '1' in column_name and column_name != "Cell_1aFan fanOnOff":
        replacement_column = column_name.replace('1', '2')
        df.loc[condition, column_name] = df.loc[condition, replacement_column]

# Cell_1aFan fanOnOff column was skipped in iterations so do the same with it
df.loc[condition, 'Cell_1aFan fanOnOff'] = df.loc[condition, 'Cell_2aFan fanOnOff']

# rename columns so that they're not specific to cooling tower
renaming = {}
for column_name in df.columns:
    if '1' in column_name:
        renaming[column_name] = column_name.replace('1','X')
    # also drop all cooling tower 2 columns since they have already been included when on
    elif '2' in column_name:
        df.drop(column_name, axis=1, inplace=True)
df.rename(renaming, inplace=True, axis=1)
```

<h3 style="color:#27469c">Verify if merging is a viable option</h3>

```
def knn_comparison(data, k, X, Y, category='whichCoolingTower'):
    x = data[[X, Y]].values
    y = data[category].astype(int).values
    clf = neighbors.KNeighborsClassifier(n_neighbors=k)
    clf.fit(x, y)
    # Plotting decision region
    plt.figure(figsize=(5,3))
    plotting.plot_decision_regions(x, y, clf=clf, legend=2)
    # Adding axes annotations
    plt.xlabel(X)
    plt.ylabel(Y)
    plt.title('Knn with K='+ str(k))
    plt.show()

for i in [5,30,80]:
    knn_comparison(data=df, k=i, X='ESB_Tower_X enteringWaterTemp', Y='ESB_Tower_X efficiency')
```

`df.isna().sum()`

```
# save as combined towers data for use in variable comparison
df.to_csv(f'{datapath}/esb/esb_merged.csv')
```